In [1]:
#Read in necessary libraries
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Read in tweets and make train, validation, and test sets
csv = 'cleantweets.csv'
my_df = pd.read_csv(csv,index_col=0)

C:\Users\Anata\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
#Sample from 1.5m tweets dataset and start to develop basic neural network.
my_df_10k = my_df.sample(n = 10000, replace = False)
#Examine it
my_df_10k.describe()
my_df_10k.target.value_counts()

0    5021
1    4979
Name: target, dtype: int64

In [4]:
x = my_df_10k.text
y = my_df_10k.target
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.1, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),(len(x_train[y_train == 0]) / (len(x_train)*1.))*100,(len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation), (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100, (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test), (len(x_test[y_test == 0]) / (len(x_test)*1.))*100, (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 9000 entries with 50.11% negative, 49.89% positive
Validation set has total 500 entries with 51.80% negative, 48.20% positive
Test set has total 500 entries with 50.40% negative, 49.60% positive


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec1 = TfidfVectorizer(max_features=90000,ngram_range=(1, 3))
tvec1.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=90000, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [6]:
x_train_tfidf = tvec1.transform(x_train)

In [7]:
x_validation_tfidf = tvec1.transform(x_validation).toarray()

In [8]:
seed = 7
np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [9]:
def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            counter=0

In [10]:
%%time
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=90000))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adamax',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
282/281 [==============================] - 70s 248ms/step - loss: 0.5823 - acc: 0.7009 - val_loss: 0.5277 - val_acc: 0.7500
Epoch 2/5
282/281 [==============================] - 72s 255ms/step - loss: 0.1764 - acc: 0.9389 - val_loss: 0.6804 - val_acc: 0.7580
Epoch 3/5
282/281 [==============================] - 74s 262ms/step - loss: 0.0410 - acc: 0.9885 - val_loss: 0.8169 - val_acc: 0.7480
Epoch 4/5
282/281 [==============================] - 78s 276ms/step - loss: 0.0168 - acc: 0.9951 - val_loss: 0.9442 - val_acc: 0.7380
Epoch 5/5
282/281 [==============================] - 77s 274ms/step - loss: 0.0099 - acc: 0.9973 - val_loss: 1.0000 - val_acc: 0.7400
Wall time: 6min 11s
